<a href="https://colab.research.google.com/github/VoxEssentia/SpeechRecongnitionWithBrython/blob/main/Lady_AI_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import pandas as pd
import random
import os
import shutil
from google.colab import files
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [3]:
model_path = "trained_rpg_ai"
model_name = "t5-small"

In [4]:
reset_model = input("🔄 Deseja resetar o modelo e treinar do zero? (s/n): ").strip().lower()
if reset_model == "s" and os.path.exists(model_path):
    print("🗑️ Removendo modelo treinado anterior...")
    shutil.rmtree(model_path)
    print("✅ Modelo resetado. Será carregado um modelo pré-treinado.")

🔄 Deseja resetar o modelo e treinar do zero? (s/n): n


In [5]:
if os.path.exists(model_path):
    print("🔄 Carregando modelo treinado...")
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)
else:
    print("📥 Carregando modelo pré-treinado...")
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

📥 Carregando modelo pré-treinado...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
def carregar_dataset():
    """Faz upload de um arquivo e carrega o dataset de treinamento."""
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]
    print(f"📂 Arquivo carregado: {file_name}")


    try:
        df = pd.read_excel(file_name)
        print("📊 Primeiras linhas do dataset:")
        print(df.head())
        data = df.to_dict(orient="records")
        return data
    except Exception as e:
        print(f"❌ Erro ao ler o arquivo: {e}")
        return []

In [7]:
def categorizar_acao(entrada):
    """Classifica a ação do PJ em categorias e subcategorias."""
    prompt = f"Ação: {entrada}\nCategoria:"
    entrada_tokenizada = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=100)
    with torch.no_grad():
        saida_ids = model.generate(
            **entrada_tokenizada,
            max_length=15,
            do_sample=False
        )
    categoria = tokenizer.decode(saida_ids[0], skip_special_tokens=True)
    return categoria.strip()

In [8]:
def fine_tune_model(data, model, tokenizer, optimizer, epochs=30, batch_size=16):
    """Treina o modelo do zero ou continua treinamento, priorizando aprendizado estável."""
    model.train()

    for epoch in range(epochs):
        random.shuffle(data)
        loss_total = 0

        for i in range(0, len(data), batch_size):
            batch = data[i:i+batch_size]
            inputs = [f"Ação: {ex['Ação']}\nCategoria:" for ex in batch]
            outputs = [ex['Classificação'] for ex in batch]

            entrada_tokenizada = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True, max_length=100)
            saida_tokenizada = tokenizer(outputs, return_tensors="pt", padding=True, truncation=True, max_length=15)

            optimizer.zero_grad()
            outputs = model(**entrada_tokenizada, labels=saida_tokenizada['input_ids'])
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            loss_total += loss.item()

        print(f"📉 Época {epoch + 1}/{epochs} - Loss médio: {loss_total / len(data)}")

    # Salvar modelo treinado
    model.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)
    print("💾 Modelo atualizado e salvo com sucesso!")

In [9]:
def treinar_assistente_automaticamente(data, model, tokenizer, optimizer, iteracoes=100):
    """Executa testes automáticos para validar a performance do modelo."""
    acertos = 0
    total = iteracoes
    erros = []

    for _ in range(iteracoes):
        exemplo = random.choice(data)
        entrada = exemplo['Ação']
        resposta_esperada = exemplo['Classificação']
        resposta_modelo = categorizar_acao(entrada)

        if resposta_modelo.lower() == resposta_esperada.lower():
            acertos += 1
            print(f"✅ Correto: {entrada} -> {resposta_modelo}")
        else:
            print(f"❌ Errado: {entrada} -> {resposta_modelo} (Esperado: {resposta_esperada})")
            erros.append({"Ação": entrada, "Classificação": resposta_esperada})

    print(f"\n🎯 Precisão do modelo após {iteracoes} testes: {(acertos / total) * 100:.2f}%\n")

    if erros:
        print(f"🔄 Realizando Fine-Tuning com {len(erros)} erros encontrados...")
        fine_tune_model(erros, model, tokenizer, optimizer)
        print("✅ Modelo atualizado com os erros corrigidos!")

In [10]:
data = carregar_dataset()

Saving dataset_10000.xlsx to dataset_10000.xlsx
📂 Arquivo carregado: dataset_10000.xlsx
📊 Primeiras linhas do dataset:
               PJ3 dispara um dardo venenoso em PNJ4  Ataque à distância
0  PJ1 usa uma besta para disparar um virote cont...  Ataque à distância
1              PJ3 lança uma faca na direção de PNJ7  Ataque à distância
2                 PJ1 dispara uma flecha contra PNJ5  Ataque à distância
3              PJ3 dispara um dardo venenoso em PNJ4  Ataque à distância
4  PJ1 usa uma besta para disparar um virote cont...  Ataque à distância


In [11]:
if not data:
    print("⚠️ Nenhum dado carregado. Verifique se o arquivo está correto.")
else:
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    treinar_assistente_automaticamente(data, model, tokenizer, optimizer)

KeyError: 'Ação'